In [39]:
import requests
from bs4 import BeautifulSoup
import time
import os
import re
import urllib
import sklearn
import pandas as pd
import tkinter

# Final Version

In [38]:
#Create date directory that're not identical
def create_date_folder(date):
    date_name = date.text.strip().replace('/', '.') # '/'  will result in error
    if not os.path.exists(date_name):
        os.makedirs(date_name)
    return

# Return a list of image_urls in a article
def get_images_urls(article_url):
    res_atc = verify_18(article_url)
    soup = BeautifulSoup(res_atc.text, 'html.parser')
    links = soup.select('div.bbs-screen.bbs-content a')
    img_urls = []
    for link in links:
        #In order not to download pics in the class : "push"
        if "ptt.cc" in link['href']:
            break
        if re.match('^https?://(i.)?(m.)?imgur.com', link['href']):
            img_urls.append(link['href'])
    return img_urls

#Create folder for each article and save pics inside
def save_images(img_urls, date, article_title):
    # "[正妹]" 這兩個字 不用存; 放在日期資料夾裡面
    dir_name = date.text.strip().replace('/', '.') + '/' + article_title[4:].strip()
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    #modify the url into the format of 'i.imgur'
    for img_url in img_urls:
        if img_url.split('//')[1].startswith('m.'):
            img_url = img_url.replace('//m.', '//i.')
        if not img_url.split('//')[1].startswith('i.'):
            img_url = img_url.split('//')[0] + '//i.' + img_url.split('//')[1]
        if not img_url.endswith('.jpg'):
            img_url += '.jpg'
        file_name = img_url.split('/')[-1]
        file_path = os.path.join(dir_name, file_name)
        #To prevent from downloading dulpicate
        if not os.path.exists(file_path):
            try:
                urllib.request.urlretrieve(img_url, file_path)
            except:
                print(img_url)
    return

Ptt_url = 'https://www.ptt.cc'
Web_url="https://www.ptt.cc/bbs/Beauty/index.html"
page = input("How many pages do you want to download? ")

#Verify that I'm over 18
def verify_18(url):
    verify_data = {'from':url, 'yes':'yes'}
    rs = requests.session()
    res = rs.post('https://www.ptt.cc/ask/over18', verify=False, data = verify_data)
    res = rs.get(url)
    return res

for i in range(int(page)):
    #res -> class
    res = verify_18(Web_url)
    #soup -> 使用BeautifulSoup讀取網頁
    soup = BeautifulSoup(res.text, 'html.parser')
    ## 網頁上的每一篇貼文都是由 <div class=”r-ent”> 的區塊包圍起來，裡面分別由
    #<div class=”nrec”> 區塊顯示推文數
    #<div class=”title”> 區塊及 <a> 區塊顯示文章連結及文章標題
    #<div class=”date”> 區塊顯示發文日期
    articles_name, pushs, dates = soup.select('div.title a'), soup.select('div.nrec'), soup.select('div.date')
    #url of next page
    paging = soup.select('div.btn-group-paging a')
    next_url = 'https://www.ptt.cc'+paging[1]['href']
    Web_url=next_url
    num_images = []
    num_pushes = []
    for article, push, date in zip(articles_name, pushs, dates):
        #只下載標籤為“正妹”之貼文
        if article.text[1:3] == "正妹":
            create_date_folder(date)
            img_urls = get_images_urls(Ptt_url + article['href'])
            num_images.append(len(img_urls))
            save_images(img_urls, date, article.text)          
            #不考慮0推或推爆的
            if push.text:
                if push.text.startswith('X') or push.text == "爆":
                    num_pushes.append(int(0))
                else:
                    num_pushes.append(int(push.text))
            else:
                num_pushes.append(int(0))
            print(num_pushes)
            print('Finished!', push.text, article.text, 'https://www.ptt.cc'+article['href'],  date.text)

How many pages do you want to download? 2
[3]
Finished! 3 [正妹] 郭思敏 https://www.ptt.cc/bbs/Beauty/M.1575787775.A.609.html 12/08
[3, 17]
Finished! 17 [正妹] 跟vv一起運動 https://www.ptt.cc/bbs/Beauty/M.1575793866.A.931.html 12/08
[3, 17, 2]
Finished! 2 [正妹] 幾秒小短片兩位姊妹 https://www.ptt.cc/bbs/Beauty/M.1575808222.A.B31.html 12/08
https://i.imgur.com/undefined.jpg
[3, 17, 2, 25]
Finished! 25 [正妹] 在俄羅斯待了三年....part5 https://www.ptt.cc/bbs/Beauty/M.1575809588.A.AB5.html 12/08
[3, 17, 2, 25, 0]
Finished!  [正妹] 週末福利 https://www.ptt.cc/bbs/Beauty/M.1575811660.A.8A1.html 12/08
[3, 17, 2, 25, 0, 6]
Finished! 6 [正妹] 屋馬文心店親切正妹店員 https://www.ptt.cc/bbs/Beauty/M.1575813104.A.358.html 12/08
[3, 17, 2, 25, 0, 6, 0]
Finished!  [正妹] 今天不想吃太鹹 https://www.ptt.cc/bbs/Beauty/M.1575813706.A.8D0.html 12/08
[3, 17, 2, 25, 0, 6, 0, 0]
Finished!  [正妹] gugudan 金世正 https://www.ptt.cc/bbs/Beauty/M.1575815360.A.731.html 12/08
[3, 17, 2, 25, 0, 6, 0, 0, 14]
Finished! 14 [正妹] 愛跳舞的16歲 https://www.ptt.cc/bbs/Beauty/M.1575815922.A.7B